# Day 1: Inverse Captcha

The captcha requires you to review a sequence of digits (your puzzle input) and find the sum of all digits that match the next digit in the list. The list is circular, so the digit after the last digit is the first digit in the list.

For example:

- `1122` produces a sum of `3` (`1` + `2`) because the first digit (`1`) matches the second digit and the third digit (`2`) matches the fourth digit.
- `1111` produces `4` because each digit (all `1`) matches the next.
- `1234` produces `0` because no digit matches the next.
- `91212129` produces `9` because the only digit that matches the next one is the last digit, `9`.

What is the solution to your captcha?

In [37]:
def checksum_next(digits):
    return sum([int(digit) for (digit, next_digit) 
                in zip(digits, digits[1:]+digits[0]) 
                if digit == next_digit])

In [38]:
[checksum_next(digits) for digits in ["1122", "1111", "1234", "91212129"]]

[3, 4, 0, 9]

## Part Two

Now, instead of considering the next digit, it wants you to consider the digit halfway around the circular list. That is, if your list contains 10 items, only include a digit in your sum if the digit `10/2 = 5` steps forward matches it. Fortunately, your list has an even number of elements.

For example:

- `1212` produces `6`: the list contains `4` items, and all four digits match the digit `2` items ahead.
- `1221` produces `0`, because every comparison is between a `1` and a `2`.
- `123425` produces `4`, because both `2`s match each other, but no other digit has a match.
- `123123` produces `12`.
- `12131415` produces `4`.

In [24]:
def checksum_rotated(digits):
    offset = len(digits) // 2
    rotated_digits = digits[offset:] + digits[:offset]
    return sum([int(digit) for (digit, matching_digit) 
         in zip(digits, rotated_digits) 
         if digit == matching_digit])

In [25]:
[checksum_rotated(digits) for digits in ["1212", "1221", "123425", "123123", "12131415"]]


[6, 0, 4, 12, 4]

# Day 2: Corruption Checksum

The spreadsheet consists of rows of apparently-random numbers. To make sure the recovery process is on the right track, they need you to calculate the spreadsheet's checksum. For each row, determine the difference between the largest value and the smallest value; the checksum is the sum of all of these differences.

For example, given the following spreadsheet:

```
5 1 9 5
7 5 3
2 4 6 8
```

- The first row's largest and smallest values are `9` and `1`, and their difference is `8`.
- The second row's largest and smallest values are `7` and `3`, and their difference is `4`.
- The third row's difference is `6`.

In this example, the spreadsheet's checksum would be `8 + 4 + 6 = 18`.


In [28]:
def checksum_spreadsheet(input):
    return sum([max(row) - min(row) for row in [[int(cell) for cell in line.split()] for line in input.split("\n")]])
    

In [31]:
checksum_spreadsheet("""5 1 9 5
7 5 3
2 4 6 8""")


18

## Part Two

It sounds like the goal is to find the only two numbers in each row where one evenly divides the other - that is, where the result of the division operation is a whole number. They would like you to find those numbers on each line, divide them, and add up each line's result.

For example, given the following spreadsheet:

```
5 9 2 8
9 4 7 3
3 8 6 5
```

- In the first row, the only two numbers that evenly divide are `8` and `2`; the result of this division is `4`.
- In the second row, the two numbers are `9` and `3`; the result is `3`.
- In the third row, the result is `2`.

In this example, the sum of the results would be `4 + 3 + 2 = 9`.



In [77]:
def checksum_spreadsheet_divisible(input):
    return sum([b // a   for row in [sorted([int(cell) for cell in line.split()]) for line in input.split("\n")] for (a,b) in combinations(row, 2) if b % a == 0])

In [78]:
checksum_spreadsheet_divisible("""5 9 2 8
9 4 7 3
3 8 6 5""")

9

## Day 3: Spiral Memory

Each square on the grid is allocated in a spiral pattern starting at a location marked 1 and then counting up while spiraling outward. For example, the first few squares are allocated like this:

```
17  16  15  14  13
18   5   4   3  12
19   6   1   2  11
20   7   8   9  10
21  22  23---> ...
```

While this is very space-efficient (no squares are skipped), requested data must be carried back to square `1` (the location of the only access port for this memory system) by programs that can only move up, down, left, or right. They always take the shortest path: the Manhattan Distance between the location of the data and square `1`.

For example:

- Data from square `1` is carried `0` steps, since it's at the access port.
- Data from square `12` is carried `3` steps, such as: down, left, left.
- Data from square `23` is carried only `2` steps: up twice.
- Data from square `1024` must be carried `31` steps.



In [326]:
from math import floor,ceil,sqrt

def dist(n):
    layer = floor(ceil(sqrt(n))/2)
    if (layer == 0):
        return 0
    
    square = (2*layer - 1) ** 2
    rest = n - square
    length = 2*layer
    side = ceil(rest / length) - 1
    middle = side * length + layer + square
    return abs(n - middle) + layer

[(a, dist(a)) for a in [1, 12, 23, 1024]]


[(1, 0), (12, 3), (23, 2), (1024, 31)]

## Part Two

So, the first few squares' values are chosen as follows:

- Square `1` starts with the value `1`.
- Square `2` has only one adjacent filled square (with value `1`), so it also stores `1`.
- Square `3` has both of the above squares as neighbors and stores the sum of their values, `2`.
- Square `4` has all three of the aforementioned squares as neighbors and stores the sum of their values, `4`.
- Square `5` only has the first and fourth squares as neighbors, so it gets the value `5`.

Once a square is written, its value does not change. Therefore, the first few squares would receive the following values:

```
147  142  133  122   59
304    5    4    2   57
330   10    1    1   54
351   11   23   25   26
362  747  806--->   ...
```


Lookup result in https://oeis.org/A141481/b141481.txt

# Day 4: High-Entropy Passphrases

A new system policy has been put in place that requires all accounts to use a passphrase instead of simply a password. A passphrase consists of a series of words (lowercase letters) separated by spaces.

To ensure security, a valid passphrase must contain no duplicate words.

For example:

- `aa bb cc dd ee` is valid.
- `aa bb cc dd aa` is not valid - the word `aa` appears more than once.
- `aa bb cc dd aaa` is valid - `aa` and `aaa` count as different words.



In [329]:
def is_valid_passphrase(phrase):
    words = phrase.split()
    return len(words) == len(set(words))

[phrase for phrase in ["aa bb cc dd ee",
                       "aa bb cc dd aa", 
                       "aa bb cc dd aaa"] 
 if is_valid_passphrase(phrase)]

['aa bb cc dd ee', 'aa bb cc dd aaa']

# Part Two

For added security, yet another system policy has been put in place. Now, a valid passphrase must contain no two words that are anagrams of each other - that is, a passphrase is invalid if any word's letters can be rearranged to form any other word in the passphrase.

For example:

- `abcde fghij` is a valid passphrase.
- `abcde xyz ecdab` is not valid - the letters from the third word can be rearranged to form the first word.
- `a ab abc abd abf abj` is a valid passphrase, because all letters need to be used when forming another word.
- `iiii oiii ooii oooi oooo` is valid.
- `oiii ioii iioi iiio` is not valid - any of these words can be rearranged to form any other word.


In [343]:
def is_still_valid_passphrase(phrase):
    sorted_words = [" ".join(sorted(word)) for word in phrase.split()]
    return len(sorted_words) == len(set(sorted_words))
[phrase for phrase in ["abcde fghij",
                        "abcde xyz ecdab",
                        "a ab abc abd abf abj",
                        "iiii oiii ooii oooi oooo", 
                        "oiii ioii iioi iiio"]
if is_still_valid_passphrase(phrase)]

['abcde fghij', 'a ab abc abd abf abj', 'iiii oiii ooii oooi oooo']

# Day 5: A Maze of Twisty Trampolines, All Alike

An urgent interrupt arrives from the CPU: it's trapped in a maze of jump instructions, and it would like assistance from any programs with spare cycles to help find the exit.

The message includes a list of the offsets for each jump. Jumps are relative: -1 moves to the previous instruction, and 2 skips the next one. Start at the first instruction in the list. The goal is to follow the jumps until one leads outside the list.

In addition, these instructions are a little strange; after each jump, the offset of that instruction increases by 1. So, if you come across an offset of 3, you would move three instructions forward, but change it to a 4 for the next time it is encountered.

For example, consider the following list of jump offsets:

```
0
3
0
1
-3
```

Positive jumps ("forward") move downward; negative jumps move upward. For legibility in this example, these offset values will be written all on one line, with the current instruction marked in parentheses. The following steps would be taken before an exit is found:

- `(0) 3  0  1  -3`  - before we have taken any steps.
- `(1) 3  0  1  -3`  - jump with offset `0` (that is, don't jump at all). Fortunately, the instruction is then incremented to `1`.
- `2 (3) 0  1  -3` - step forward because of the instruction we just modified. The first instruction is incremented again, now to `2`.
- `2  4  0  1 (-3)` - jump all the way to the end; leave a `4` behind.
- `2 (4) 0  1  -2`  - go back to where we just were; increment `-3` to `-2`.
- `2  5  0  1  -2`  - jump `4` steps forward, escaping the maze.

In this example, the exit is reached in `5` steps.

In [369]:
def steps(jumps):
    position = 0
    count = 0
    length = len(jumps)
    while 0 <= position < length:
        offset = jumps[position]
        jumps[position] = offset + 1
        position += offset
        count += 1
    return count
    
input = [0,3,0,1,-3]    
steps(input), input


(5, [2, 5, 0, 1, -2])

## Part Two

Now, the jumps are even stranger: after each jump, if the offset was three or more, instead decrease it by `1`. Otherwise, increase it by `1` as before.

Using this rule with the above example, the process now takes `10` steps, and the offset values after finding the exit are left as `2 3 2 3 -1`.


In [374]:
def crazy_steps(jumps):
    position = 0
    count = 0
    length = len(jumps)
    while 0 <= position < length:
        offset = jumps[position]
        jumps[position] = offset - 1 if offset >= 3 else offset + 1
        position += offset
        count += 1
    return count

input = [0,3,0,1,-3]
crazy_steps(input), input

(10, [2, 3, 2, 3, -1])

In [377]:
input = """2
0
0
1
2
0
1
-4
-5
0
-1
0
-6
0
-5
2
-9
-11
-15
-3
-11
-12
-14
-5
-16
-3
-13
-6
0
-3
-17
0
-17
-5
-1
-26
-21
-14
-20
-7
-24
-26
-32
-41
-2
-18
-18
-13
-28
0
-32
-3
-2
-14
-17
-54
-22
-34
-33
-34
0
-46
-3
-44
-58
-10
-56
-65
-46
-24
-20
-4
-27
-41
-33
-31
-20
-75
-73
-41
-36
-31
-70
-46
-1
-79
-61
-51
-77
-44
-55
-2
-18
-26
-50
-79
-59
-69
-62
-80
-13
-69
-97
-71
-24
-7
-40
-10
-23
-85
-97
-103
-55
-90
-40
-60
-98
-95
-39
-76
-63
-12
-2
-65
-109
-47
-12
-37
-5
-23
-125
-124
-49
-91
-70
-134
-54
-122
-135
-12
-23
-22
-83
-40
-133
-77
-88
-119
-146
-26
-12
-108
-63
-111
-148
-99
-77
-77
-76
-89
-37
-95
-105
-76
-137
-151
-146
-141
-162
-12
-68
-36
-116
-60
-73
-61
-60
-101
-168
-142
-143
-118
-165
-108
-179
-180
-11
-152
-67
-33
-10
-169
-155
-16
-136
-165
-164
2
1
-28
-131
-86
-153
-116
-113
-149
-66
-64
-36
-168
-116
-159
-15
-180
-125
-146
-135
-105
-161
-133
-207
-192
-192
-99
-146
-93
-21
-5
-189
-86
-16
-4
-44
-167
-20
-201
-110
-103
-223
-182
-71
-194
-68
-90
-237
-147
2
-88
-184
-90
-12
-119
-85
-138
-179
-152
-158
-82
-122
-179
-191
-120
-174
-165
-137
-181
-58
-250
-66
-194
-202
-171
-179
-137
-250
-248
-167
-108
-27
-175
-34
-254
-35
-157
-158
-31
-52
-236
-238
-247
-279
-209
-257
-167
-151
-7
-182
-2
-149
-87
-245
-141
-238
-186
-71
-97
-128
-147
-52
-93
-142
-197
-296
-73
-89
-14
-253
-190
-295
-312
-47
-236
-233
-238
-305
-121
-191
-251
-91
-307
-77
-228
-300
-197
-91
-191
-299
-77
-255
-51
-232
-64
-198
-187
-96
-86
-203
-216
-203
-343
-203
-78
-99
-174
-269
-349
-173
-52
-233
-154
-151
-304
-70
-235
-106
-226
-325
-142
-192
-115
-170
-15
-35
-174
-267
-108
-374
-128
-60
-131
-364
-371
-56
-96
-365
-305
-140
-50
-220
-179
-43
-356
-120
-216
-276
-103
-389
-28
-393
-341
-74
-85
-361
-68
-111
-4
-216
-263
-115
-194
-382
-285
-176
-145
-24
-59
-291
-170
-358
-226
-355
-292
-185
-297
-156
-248
-332
-319
-311
-46
-170
-428
-222
-35
-136
-206
-81
-330
-89
-75
-248
-42
-52
-24
-39
-129
-228
-242
-396
-222
-433
-168
-362
-4
-345
-395
-435
-14
-439
-136
-267
-417
-107
-177
-8
-208
-219
-222
-453
-155
-183
-252
0
-173
-71
-164
-187
-80
-292
-246
-89
-217
-227
-93
-244
-82
-51
-23
-283
-261
-50
-384
-415
-149
-103
-481
-404
-267
-80
-61
-130
-228
-310
-319
-186
-88
-173
-40
-69
-231
-398
-342
-176
-33
-304
-232
-220
-381
-436
-74
-116
-398
-467
-341
-483
-137
-5
-437
-67
-296
-137
-166
-216
-192
-307
-68
-319
-296
-524
-308
-68
-21
-515
-531
-221
-173
-261
-200
-450
-95
-366
-14
-29
-23
-173
-397
-373
-283
-104
-246
-153
-240
-378
-306
-495
-518
-459
-459
-340
-475
-96
-347
-8
-365
-7
-482
-113
-223
-313
-456
-89
-205
-507
-538
-115
-310
-484
-96
-367
-582
-32
-550
-247
-257
-479
-165
-346
-514
-188
-180
-506
-117
-92
-128
-507
-387
-52
-535
-210
-221
-560
-245
-70
-552
-99
-529
-607
-263
-345
-253
-426
-351
-92
-489
-478
-226
-606
-287
-277
-432
-336
-418
-94
-2
-192
-600
-454
-26
-3
-630
-294
-105
-439
-589
-425
-623
-451
-487
-117
-538
-78
-126
-485
-326
-455
-370
-389
-379
-158
-441
-524
-435
-160
-198
-172
-313
-380
-128
-166
-562
-427
-23
-616
-95
-188
-417
-419
-589
-488
-377
-520
-412
-348
-359
-488
-108
-409
-56
-460
-364
-233
-352
-59
-313
-609
-534
-432
-104
-514
-68
-83
-305
-308
-645
-535
-624
-453
-630
-274
-98
-280
-38
-443
-620
-411
-624
-379
-373
-338
-410
-382
-171
-645
-430
-294
-696
-513
-659
-690
-558
-2
-325
-234
-437
-610
-158
-186
-539
-405
-78
-100
-311
-201
-558
-604
-386
-457
-125
-419
-680
-147
-237
-107
-155
-550
-565
-214
-528
-353
-637
-6
-634
-332
-92
-474
-289
-617
-141
-398
-367
-537
-369
-88
-608
-699
-257
-602
-276
-406
-92
-746
-398
-387
-234
-331
-225
-480
-667
-264
-299
-673
-265
-142
-512
-573
-508
-551
-471
-270
-328
-648
-625
-779
-232
-393
-749
-84
-240
-59
-220
-55
-224
-350
-130
-23
-239
-105
-2
-762
-702
-163
-94
-350
-11
-176
-43
-654
-136
-348
-215
-67
-599
-757
-636
-367
-61
-209
-623
-342
-111
-93
-14
-613
-362
-837
-734
-468
-803
-548
-699
-744
-429
-243
-633
-382
-780
-668
-498
-664
-539
-781
-525
-697
-715
-126
-276
-504
-175
-592
-688
-92
-548
-298
-33
-532
-674
-57
-531
-488
-310
-90
-757
-496
-132
-733
-701
-61
-797
-215
-319
-700
-295
-572
-41
-140
-176
-479
-560
-164
-338
-794
-132
-453
-709
-445
-802
2
-336
-562
-802
-878
-547
-368
-502
-574
-275
-687
-560
-432
-423
-174
-367
-59
-605
-340
-626
-142
-601
-488
-299
-466
-521
-783
-140
-731
-779
-252
-663
-906
-410
-601
-524
-332
-750
-556
-730
-749
-294
-798
-93
-345
-316
-186
-634
-904
-237
-134
-765
-953
-170
-854
-910
-99
-782
-564
-505
-49
-827
-64
-297
-548
-841
-598
-414
-184
-67
-99
-880
-855
-722
-725
-582
-416
-473
-339
-491
-162
-311
-43
-938
-608
-524
-212
-4
-945
-544
-879
-382
-21
-512
-169
-284
-140
-588
-407
-56
-610
-75
-412
-321
-801
-881
-220
-388
-116
-962
-1007
-862
-20
-409
-116
-943
-558
-1001
-548
-302
-165
-730
-1012
-669
-875
-393
-979""".split("\n")
crazy_steps([int(line) for line in input])

24568703